In [7]:
import pandas as pd

spread_df = pd.read_csv("outputs/spreads.csv", parse_dates=["Date"])
spread_df.set_index("Date", inplace=True)

chrono_df = pd.read_csv("outputs/chrono_dummy.csv", parse_dates=["Date"])
chrono_df.set_index("Date", inplace=True)

bert_df = pd.read_csv("outputs/bert_dummy.csv", parse_dates=["Date"])
bert_df.set_index("Date", inplace=True)

traditional_df = pd.read_csv("outputs/traditional_dummy.csv", parse_dates=["Date"])
traditional_df.set_index("Date", inplace=True)


spread_df = spread_df.merge(chrono_df, how="inner", on=["Date", "Ticker Pair"], validate='one_to_one')
spread_df = spread_df.merge(bert_df, how="inner", on=["Date", "Ticker Pair"], validate='one_to_one')
spread_df = spread_df.merge(traditional_df, how="inner", on=["Date", "Ticker Pair"], validate='one_to_one')

print(spread_df.head(10))

           Ticker Pair     Spread  CHRONOBERT Spread CHRONOBERT Position  \
Date                                                                       
2019-02-28    AMD-NVDA  19.700950          20.694379               Short   
2019-03-31    AMD-NVDA  21.062952          20.786423                Long   
2019-04-30    AMD-NVDA  23.137207          24.432584               Short   
2019-05-31    AMD-NVDA  24.043767          27.089826               Short   
2019-06-30    AMD-NVDA  26.288822          25.820515                Long   
2019-07-31    AMD-NVDA  26.257243          25.788969                Long   
2019-08-31    AMD-NVDA  27.283192          30.441618               Short   
2019-09-30    AMD-NVDA  24.660011          26.194881               Short   
2019-10-31    AMD-NVDA  28.929632          27.990683                Long   
2019-11-30    AMD-NVDA  33.754620          34.839740               Short   

            BERT Spread BERT Position  Traditional Spread Traditional Position  
Date  